# Porto Seguro ML

Porto Seguro, one of Brazil’s largest auto and homeowner insurance companies, completely agrees. Inaccuracies in car insurance company’s claim predictions raise the cost of insurance for good drivers and reduce the price for bad ones.

In the [Porto Seguro Safe Driver Prediction competition](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction), the challenge is to **build a model that predicts the probability that a driver will initiate an auto insurance claim in the next year**.
While Porto Seguro has used machine learning for the past 20 years, they’re looking to Kaggle’s machine learning community to explore new, more powerful methods. A more accurate prediction will allow them to further tailor their prices, and hopefully make auto insurance coverage more accessible to more drivers.

Lucky for you, a machine learning model was built to solve the Porto Seguro problem by the data scientist on your team. The solution notebook has steps to load data, split the data into test and train sets, train, evaluate and save a LightGBM model that will be used for the future challenges.

#### Hint: use shift + enter to run the code cells below. Once the cell turns from [*] to [#], you can be sure the cell has run. 

In [1]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

## Import Needed Packages

Import the packages needed for this solution notebook. The most widely used packages for machine learning for [scikit-learn](https://scikit-learn.org/stable/), [pandas](https://pandas.pydata.org/docs/getting_started/index.html#getting-started), and [numpy](https://numpy.org/). These packages have various features, as well as a lot of clustering, regression and classification algorithms that make it a good choice for data mining and data analysis. In this notebook, we're using a training function from [lightgbm](https://lightgbm.readthedocs.io/en/latest/index.html).

In [2]:
import os
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
import joblib
from sklearn import metrics

## Load Data

Load the training dataset from the ./data/ directory. Df.shape() allows you to view the dimensions of the dataset you are passing in. If you want to view the first 5 rows of data, df.head() allows for this.

In [3]:
DATA_DIR = "./data"
data_df = pd.read_csv(os.path.join(DATA_DIR, 'porto_seguro_safe_driver_prediction_input.csv'))

In [4]:
!ls ./data/porto_seguro_safe_driver_prediction_input.csv -l

-rwxrwxrwx 1 root root 115852544 Mar  9 10:19 ./data/porto_seguro_safe_driver_prediction_input.csv


In [5]:
data_df.shape

(595212, 59)

In [6]:
data_df.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


## Split Data into Train and Validatation Sets

Partitioning data into training, validation, and holdout sets allows you to develop highly accurate models that are relevant to data that you collect in the future, not just the data the model was trained on. 

In machine learning, features are the measurable property of the object you’re trying to analyze. Typically, features are the columns of the data that you are training your model with minus the label. In machine learning, a label (categorical) or target (regression) is the output you get from your model after training it.

In [7]:
features = data_df.drop(['target', 'id'], axis = 1)
labels = np.array(data_df['target'])

features_train, features_valid, labels_train, labels_valid = train_test_split(features, labels, 
                                                                              test_size=0.2, 
                                                                              random_state=0)

train_data = lightgbm.Dataset(features_train, label=labels_train)
valid_data = lightgbm.Dataset(features_valid, label=labels_valid, free_raw_data=False)

## Train Model

A machine learning model is an algorithm which learns features from the given data to produce labels which may be continuous or categorical ( regression and classification respectively ). In other words, it tries to relate the given data with its labels, just as the human brain does.

In this cell, the data scientist used an algorithm called [LightGBM](https://lightgbm.readthedocs.io/en/latest/), which primarily used for unbalanced datasets. AUC will be explained in the next cell.

In [8]:
parameters = {
    'learning_rate': 0.02,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'sub_feature': 0.7,
    'num_leaves': 60,
    'min_data': 100,
    'min_hessian': 1,
    'verbose': 4
}


model = lightgbm.train(parameters,
                           train_data,
                           valid_sets=valid_data,
                           num_boost_round=500,
                           early_stopping_rounds=20)


[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.872018
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.357386
[LightGBM] [Debug] init for col-wise cost 0.038400 seconds, init for row-wise cost 0.137464 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.054393 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1365
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 57
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 12
[1]	valid_0's auc: 0.600139
Training until validation scores don

[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 15
[76]	valid_0's auc: 0.632179
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 14
[77]	valid_0's auc: 0.63216
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 16
[78]	valid_0's auc: 0.632133
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 10
[79]	valid_0's auc: 0.632202
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 10
[80]	valid_0's auc: 0.632235
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 20
[81]	valid_0's auc: 0.632427
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 11
[82]	valid_0's auc: 0.63249
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 16
[83]	valid_0's auc: 0.632594
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 11
[84]	valid_0's auc: 0.632657
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 11
[85]	valid_0's auc: 0.632768
[LightGBM] [

[158]	valid_0's auc: 0.636677
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 12
[159]	valid_0's auc: 0.636701
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 13
[160]	valid_0's auc: 0.636705
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 17
[161]	valid_0's auc: 0.63675
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 17
[162]	valid_0's auc: 0.636781
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 16
[163]	valid_0's auc: 0.636831
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 11
[164]	valid_0's auc: 0.636912
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 15
[165]	valid_0's auc: 0.637023
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 13
[166]	valid_0's auc: 0.637008
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 13
[167]	valid_0's auc: 0.637045
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 12
[

[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 14
[241]	valid_0's auc: 0.638359
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 16
[242]	valid_0's auc: 0.638372
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 18
[243]	valid_0's auc: 0.638431
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 14
[244]	valid_0's auc: 0.638433
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 21
[245]	valid_0's auc: 0.638452
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 12
[246]	valid_0's auc: 0.638493
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 13
[247]	valid_0's auc: 0.638522
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 16
[248]	valid_0's auc: 0.638478
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 10
[249]	valid_0's auc: 0.638437
[LightGBM] [Debug] Trained a tree with leaves = 60 and max_depth = 14
[250]	valid_0's auc: 0.638483


## Evaluate Model

Evaluating performance is an essential task in machine learning. In this case, because this is a classification problem, the data scientist elected to use an AUC - ROC Curve. When we need to check or visualize the performance of the multi - class classification problem, we use AUC (Area Under The Curve) ROC (Receiver Operating Characteristics) curve. It is one of the most important evaluation metrics for checking any classification model’s performance.

<img src="https://www.researchgate.net/profile/Oxana_Trifonova/publication/276079439/figure/fig2/AS:614187332034565@1523445079168/An-example-of-ROC-curves-with-good-AUC-09-and-satisfactory-AUC-065-parameters.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 12px; width: 320px; height: 239px;" />

In [9]:
predictions = model.predict(valid_data.data)
fpr, tpr, thresholds = metrics.roc_curve(valid_data.label, predictions)

model_metrics = {"auc": (metrics.auc(fpr, tpr))}
print("AUC =", model_metrics)

AUC = {'auc': 0.6388635476931616}


## Save Model
 
In machine learning, we need to save the trained models in a file and restore them in order to reuse it to compare the model with other models, to test the model on a new data. The saving of data is called Serializaion, while restoring the data is called Deserialization.

In [10]:
model_name = "portomlmodel.pkl"
joblib.dump(value=model, filename=model_name)

['portomlmodel.pkl']

In [11]:
!ls portomlmodel.pkl -l

-rwxrwxrwx 1 root root 1868549 Mar 22 12:16 portomlmodel.pkl
